In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import json
from collections import defaultdict
import codecs
from sklearn.utils import shuffle
from collections import Counter
import itertools
import re
import math
import os
import random
import sys
import time
from six.moves import xrange  # pylint: disable=redefined-builtin

In [2]:
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import ops
from tensorflow.python.util import nest
from tensorflow.python.ops import variable_scope

In [3]:
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

In [4]:
def softmax(x):
    prob = np.exp(x) / np.sum(np.exp(x), axis=0)
    return prob

In [5]:
def sequence_loss_by_example(logits, targets, weights,
                             average_across_timesteps=True,
                             softmax_loss_function=None, name=None):
    # print('sequence_loss_by_example')
    if len(targets) != len(logits) or len(weights) != len(logits):
        raise ValueError("Lengths of logits, weights, and targets must be the same "
                     "%d, %d, %d." % (len(logits), len(weights), len(targets)))
    with ops.name_scope(name, "sequence_loss_by_example",
                      logits + targets + weights):
        log_perp_list = []
        for logit, target, weight in zip(logits, targets, weights):
            if softmax_loss_function is None:
                target = array_ops.reshape(target, [-1])
                crossent = nn_ops.sparse_softmax_cross_entropy_with_logits(logit, target)
            else:
                crossent = softmax_loss_function(logit, target)
            log_perp_list.append(crossent * weight)
        log_perps = math_ops.add_n(log_perp_list)
        if average_across_timesteps:
            total_size = math_ops.add_n(weights)
            total_size += 1e-12  # Just to avoid division by 0 for all-0 weights.
            log_perps /= total_size
    return log_perps

In [6]:
def sequence_loss(logits, targets, weights,
                  average_across_timesteps=True, average_across_batch=True,
                  softmax_loss_function=None, name=None):
    #print("sequence_loss")
    with ops.name_scope(name, "sequence_loss", logits + targets + weights):
        cost = math_ops.reduce_sum(sequence_loss_by_example(
                                                        logits, targets, weights,
                                                        average_across_timesteps=average_across_timesteps,
                                                        softmax_loss_function=softmax_loss_function))
        if average_across_batch:
            batch_size = array_ops.shape(targets[0])[0]
            return cost / math_ops.cast(batch_size, cost.dtype)
        else:
            return cost

In [7]:
_BIAS_VARIABLE_NAME = "bias"
_WEIGHTS_VARIABLE_NAME = "kernel"
def linear(args,
            output_size,
            bias,
            bias_initializer=None,
            kernel_initializer=None):
    if args is None or (nest.is_sequence(args) and not args):
        raise ValueError("`args` must be specified")
    if not nest.is_sequence(args):
        args = [args]

    # Calculate the total size of arguments on dimension 1.
    total_arg_size = 0
    shapes = [a.get_shape() for a in args]
    for shape in shapes:
        if shape.ndims != 2:
            raise ValueError("linear is expecting 2D arguments: %s" % shapes)
        if shape[1].value is None:
            raise ValueError("linear expects shape[1] to be provided for shape %s, "
                       "but saw %s" % (shape, shape[1]))
        else:
            total_arg_size += shape[1].value

    dtype = [a.dtype for a in args][0]
    
    # Now the computation.
    scope = vs.get_variable_scope()
    with vs.variable_scope(scope) as outer_scope:
        weights = vs.get_variable(
                                    _WEIGHTS_VARIABLE_NAME, [total_arg_size, output_size],
                                    dtype=dtype,
                                    initializer=kernel_initializer)
        if len(args) == 1:
            res = math_ops.matmul(args[0], weights)
        else:
            res = math_ops.matmul(array_ops.concat(args, 1), weights)
        if not bias:
            return res
        with vs.variable_scope(outer_scope) as inner_scope:
            inner_scope.set_partitioner(None)
            if bias_initializer is None:
                bias_initializer = init_ops.constant_initializer(0.0, dtype=dtype)
            biases = vs.get_variable(
                                      _BIAS_VARIABLE_NAME, [output_size],
                                      dtype=dtype,
                                        initializer=bias_initializer)
        return nn_ops.bias_add(res, biases)

In [8]:
def attention_decoder(decoder_inputs,
                      initial_state,
                      attention_states,
                      cell,
                      output_size=None,
                      num_heads=1,
                      loop_function=None,
                      dtype=None,
                      scope=None,
                      initial_state_attention=False):

    #print('attention_decoder')
    if not decoder_inputs:
        raise ValueError("Must provide at least 1 input to attention decoder.")
    if num_heads < 1:
        raise ValueError("With less than 1 heads, use a non-attention decoder.")
    if attention_states.get_shape()[2].value is None:
        raise ValueError("Shape[2] of attention_states must be known: %s"
                     % attention_states.get_shape())
    if output_size is None:
        output_size = cell.output_size

    with variable_scope.variable_scope(scope or "attention_decoder", dtype=dtype) as scope:
        dtype = scope.dtype

        batch_size = array_ops.shape(decoder_inputs[0])[0]  # Needed for reshaping.
        attn_length = attention_states.get_shape()[1].value
        if attn_length is None:
            attn_length = shape(attention_states)[1]
        attn_size = attention_states.get_shape()[2].value

        # To calculate W1 * h_t we use a 1-by-1 convolution, need to reshape before.
        hidden = array_ops.reshape(attention_states, [-1, attn_length, 1, attn_size])
        hidden_features = []
        v = []
        attention_vec_size = attn_size  # Size of query vectors for attention.
        for a in xrange(num_heads):
            k = variable_scope.get_variable("AttnW_%d" % a,
                                      [1, 1, attn_size, attention_vec_size])
            hidden_features.append(nn_ops.conv2d(hidden, k, [1, 1, 1, 1], "SAME"))
            v.append(variable_scope.get_variable("AttnV_%d" % a, [attention_vec_size]))

        state = initial_state

        def attention(query):
            """Put attention masks on hidden using hidden_features and query."""
            #print("attention_decoder.attention")
            ds = []  # Results of attention reads will be stored here.
            if nest.is_sequence(query):  # If the query is a tuple, flatten it.
                query_list = nest.flatten(query)
                for q in query_list:  # Check that ndims == 2 if specified.
                    ndims = q.get_shape().ndims
                    if ndims:
                        assert ndims == 2
                query = array_ops.concat(query_list,1)
            for a in xrange(num_heads):
                with variable_scope.variable_scope("Attention_%d" % a):
                    #print('调用linear')
                    y = linear(query, attention_vec_size,True)
                    y = array_ops.reshape(y, [-1, 1, 1, attention_vec_size])
                    # Attention mask is a softmax of v^T * tanh(...).
                    s = math_ops.reduce_sum(v[a] * math_ops.tanh(hidden_features[a] + y), [2, 3])
                    a = nn_ops.softmax(s)
                    # Now calculate the attention-weighted vector d.
                    d = math_ops.reduce_sum(
                    array_ops.reshape(a, [-1, attn_length, 1, 1]) * hidden,[1, 2])
                    ds.append(array_ops.reshape(d, [-1, attn_size]))
            return ds

        outputs = []
        prev = None
        batch_attn_size = array_ops.stack([batch_size, attn_size])
        attns = [array_ops.zeros(batch_attn_size, dtype=dtype)for _ in xrange(num_heads)]
        for a in attns:  # Ensure the second shape of attention vectors is set.
            a.set_shape([None, attn_size])
        if initial_state_attention:
            attns = attention(initial_state)
        for i, inp in enumerate(decoder_inputs):
            if i > 0:
                variable_scope.get_variable_scope().reuse_variables()
            # If loop_function is set, we use it instead of decoder_inputs.
            if loop_function is not None and prev is not None:
                with variable_scope.variable_scope("loop_function", reuse=True):
                    inp = loop_function(prev, i)
            # Merge input and previous attentions into one vector of the right size.
            input_size = inp.get_shape().with_rank(2)[1]
            if input_size.value is None:
                raise ValueError("Could not infer input size from input: %s" % inp.name)
                
            #print('调用linear')
            x = linear([inp] + attns, input_size,True)
            cell_output, state = cell(x, state)
            if i == 0 and initial_state_attention:
                #print('找错1')
                with variable_scope.variable_scope(variable_scope.get_variable_scope(),
                                           reuse=True):
                    #print("找错2")
                    attns = attention(state)
            else:
                #print("找错3")
                attns = attention(state)
            
            with variable_scope.variable_scope("AttnOutputProjection"):
                #print('output = linear([cell_output] + attns, output_size,True)')
                output = linear([cell_output] + attns, output_size,True)
            if loop_function is not None:
                prev = output
            #print('outputs.append(output)')
            outputs.append(output)

    return outputs, state

In [9]:
def _argmax_or_mcsearch(embedding, output_projection=None, update_embedding=True, mc_search=False):
    def loop_function(prev, _):
        if output_projection is not None:
            prev = nn_ops.xw_plus_b(prev, output_projection[0], output_projection[1])


        if isinstance(mc_search, bool):
            prev_symbol = tf.reshape(tf.multinomial(prev, 1), [-1]) if mc_search else math_ops.argmax(prev, 1)
        else:
            prev_symbol = tf.cond(mc_search, lambda: tf.reshape(tf.multinomial(prev, 1), [-1]), lambda: tf.argmax(prev, 1))


        emb_prev = embedding_ops.embedding_lookup(embedding, prev_symbol)
        if not update_embedding:
            emb_prev = array_ops.stop_gradient(emb_prev)
        return emb_prev
    return loop_function

In [10]:
def embedding_attention_decoder(decoder_inputs,
                                initial_state,
                                attention_states,
                                cell,
                                num_symbols,
                                embedding_size,
                                num_heads=1,
                                output_size=None,
                                output_projection=None,
                                feed_previous=False,
                                update_embedding_for_previous=True,
                                dtype=None,
                                scope=None,
                                initial_state_attention=False,
                                mc_search = False):
    #print('embedding_attention_decoder')
    if output_size is None:
        output_size = cell.output_size
    if output_projection is not None:
        proj_biases = ops.convert_to_tensor(output_projection[1], dtype=dtype)
        proj_biases.get_shape().assert_is_compatible_with([num_symbols])

    with variable_scope.variable_scope(scope or "embedding_attention_decoder", dtype=dtype) as scope:
        embedding = variable_scope.get_variable("embedding",[num_symbols, embedding_size])

        loop_function = None
        if feed_previous == True:
            loop_function = _argmax_or_mcsearch(embedding, output_projection, update_embedding_for_previous, mc_search)

        emb_inp = [embedding_ops.embedding_lookup(embedding, i) for i in decoder_inputs]
        return attention_decoder(
                                emb_inp,
                                initial_state,
                                attention_states,
                                cell,
                                output_size=output_size,
                                num_heads=num_heads,
                                loop_function=loop_function,
                                initial_state_attention=initial_state_attention,
                                scope=scope)



In [11]:
# return output, state, encoder_state

def embedding_attention_seq2seq(encoder_inputs,
                                decoder_inputs,
                                cell,
                                num_encoder_symbols,
                                num_decoder_symbols,
                                embedding_size,
                                num_heads=1,
                                output_projection=None,
                                feed_previous=False,
                                dtype=None,
                                scope=None,
                                initial_state_attention=False,
                                mc_search=False):

    with variable_scope.variable_scope(scope or "embedding_attention_seq2seq", dtype=dtype) as scope:
        dtype = scope.dtype
        #print('embedding_attention_seq2seq')
        
        # Encoder.
        #print('encoder_cell')
        encoder_cell = tf.contrib.rnn.core_rnn_cell.EmbeddingWrapper(
                cell, embedding_classes=num_encoder_symbols,
                embedding_size=embedding_size)
        #print('encoder_outputs, encoder_state = tf.contrib.rnn.static_rnn')
        encoder_outputs, encoder_state = tf.contrib.rnn.static_rnn(
                encoder_cell, encoder_inputs, dtype=dtype)
        #print('encoder_outputs')
        
        # First calculate a concatenation of encoder outputs to put attention on.
        #print('top-state')
        #top_states = [array_ops.reshape([-1, 1, cell.output_size],e)for e in encoder_outputs]
        top_states = tf.stack(encoder_outputs)
        top_states = tf.transpose(top_states, [1,0,2])
        #print('attention_state')
        attention_states = array_ops.concat(top_states,1)

        # Decoder.
        #print('decoder')
        output_size = None
        if output_projection is None:
            cell = rnn_cell.OutputProjectionWrapper(cell, num_decoder_symbols)
            output_size = num_decoder_symbols

        if isinstance(feed_previous, bool):
            outputs, state = embedding_attention_decoder(
                          decoder_inputs,
                          encoder_state,
                          attention_states,
                          cell,
                          num_decoder_symbols,
                          embedding_size,
                          num_heads=num_heads,
                          output_size=output_size,
                          output_projection=output_projection,
                          feed_previous=feed_previous,
                          initial_state_attention=initial_state_attention,
                          mc_search=mc_search,
                          scope=scope)
            return outputs, state, encoder_state

        # If feed_previous is a Tensor, we construct 2 graphs and use cond.
        def decoder(feed_previous_bool):
            reuse = None if feed_previous_bool else True
            with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=reuse) as scope:
                outputs, state = embedding_attention_decoder(
                            decoder_inputs,
                            encoder_state,
                            attention_states,
                            cell,
                            num_decoder_symbols,
                            embedding_size,
                            num_heads=num_heads,
                            output_size=output_size,
                            output_projection=output_projection,
                            feed_previous=feed_previous_bool,
                            update_embedding_for_previous=False,
                            initial_state_attention=initial_state_attention,
                            mc_search=mc_search,
                            scope=scope)
                state_list = [state]
                if nest.is_sequence(state):
                    state_list = nest.flatten(state)
                return outputs + state_list

        outputs_and_state = control_flow_ops.cond(feed_previous,
                                              lambda: decoder(True),
                                              lambda: decoder(False))
        outputs_len = len(decoder_inputs)  # Outputs length same as decoder inputs.
        state_list = outputs_and_state[outputs_len:]
        state = state_list[0]
        if nest.is_sequence(encoder_state):
            state = nest.pack_sequence_as(structure=encoder_state,
                                    flat_sequence=state_list)
        return outputs_and_state[:outputs_len], state, encoder_state




In [12]:
# return outputs, losses, encoder_states

def model_with_buckets(encoder_inputs, decoder_inputs, targets, weights, buckets, vocab_size, batch_size, seq2seq,
                       output_projection=None, softmax_loss_function=None, per_example_loss=False, name=None):
    #print('model_with_buckets')
    if len(encoder_inputs) < buckets[-1][0]:
        raise ValueError("Length of encoder_inputs (%d) must be at least that of la"
                     "st bucket (%d)." % (len(encoder_inputs), buckets[-1][0]))
    if len(targets) < buckets[-1][1]:
        raise ValueError("Length of targets (%d) must be at least that of last"
                     "bucket (%d)." % (len(targets), buckets[-1][1]))
    if len(weights) < buckets[-1][1]:
        raise ValueError("Length of weights (%d) must be at least that of last"
                     "bucket (%d)." % (len(weights), buckets[-1][1]))

    all_inputs = encoder_inputs + decoder_inputs + targets + weights
    losses = []
    outputs = []
    encoder_states = []
    with ops.name_scope(name, "model_with_buckets", all_inputs):
        for j, bucket in enumerate(buckets):
            with variable_scope.variable_scope(variable_scope.get_variable_scope(),
                                         reuse=True if j > 0 else None):
                bucket_outputs, decoder_states, encoder_state = seq2seq(encoder_inputs[:bucket[0]],
                                    decoder_inputs[:bucket[1]])
                outputs.append(bucket_outputs)
                #print("bucket outputs: %s" %bucket_outputs)
                encoder_states.append(encoder_state)
                if per_example_loss:
                    losses.append(sequence_loss_by_example(
                    outputs[-1], targets[:bucket[1]], weights[:bucket[1]],
                    softmax_loss_function=softmax_loss_function))
                else:
                    # losses.append(sequence_loss_by_mle(outputs[-1], targets[:bucket[1]], vocab_size, bucket[1], batch_size, output_projection))
                    losses.append(sequence_loss(outputs[-1], targets[:bucket[1]], weights[:bucket[1]], softmax_loss_function=softmax_loss_function))

    return outputs, losses, encoder_states

In [13]:
class Seq2SeqModel(object):
    def __init__(self, config,name_scope, forward_only,use_lstm=False,num_samples=512):

        dtype=tf.float32
        
        source_vocab_size = config.vocab_size
        target_vocab_size = config.vocab_size
        size= config.emb_dim

        self.buckets = config.buckets
        self.batch_size = config.batch_size
        self.learning_rate = tf.Variable(float(config.learning_rate), trainable=False)
        self.learning_rate_decay_op = self.learning_rate.assign(
            self.learning_rate * config.learning_rate_decay_factor)
        self.global_step = tf.Variable(0, trainable=False)
        max_gradient_norm = config.max_gradient_norm
        self.forward_only = tf.placeholder(tf.bool, name="forward_only")
        #self.num_layers = config.num_layers
        num_layers = config.num_layers
        self.max_gradient_norm = config.max_gradient_norm
        
        # If we use sampled softmax, we need an output projection.
        output_projection = None
        softmax_loss_function = None
        
        # ADD
        self.mc_search = tf.placeholder(tf.bool, name="mc_search")
        self.up_reward = tf.placeholder(tf.bool, name="up_reward")
        self.reward_bias = tf.get_variable("reward_bias", [1], dtype=tf.float32)
        
        # Sampled softmax only makes sense if we sample less than vocabulary size.
        if num_samples > 0 and num_samples < target_vocab_size:
            w = tf.get_variable("proj_w", [size, target_vocab_size])
            w_t = tf.transpose(w)
            b = tf.get_variable("proj_b", [target_vocab_size])
            output_projection = (w, b)

            def sampled_loss(inputs, labels):
                
                labels = tf.reshape(labels, [-1, 1])
                local_w_t = tf.cast(w_t, tf.float32)
                local_b = tf.cast(b, tf.float32)
                local_inputs = tf.cast(inputs, tf.float32)
                
                return tf.cast(
                    tf.nn.sampled_softmax_loss(local_w_t, local_b, labels, local_inputs,
                                               num_samples, target_vocab_size), dtype)
            softmax_loss_function = sampled_loss

        # Create the internal multi-layer cell for our RNN.
        #single_cell = tf.contrib.rnn.GRUCell(size)
        single_cell = tf.contrib.rnn.LSTMCell(size)
        print('single_cell')
        if use_lstm:
            single_cell = tf.contrib.rnn.LSTMCell(size)
        cell = single_cell
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.5)
        if num_layers > 1:
            cell = tf.contrib.rnn.MultiRNNCell([single_cell] * num_layers)
        # tf.contrib.seq2seq
        # The seq2seq function: we use embedding for the input and attention.
        def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
            return embedding_attention_seq2seq(
                  encoder_inputs, decoder_inputs, cell,
                  num_encoder_symbols=source_vocab_size,
                  num_decoder_symbols=target_vocab_size,
                  embedding_size=size,
                  output_projection=output_projection,
                  feed_previous=do_decode)

        # Feeds for inputs.
        self.encoder_inputs = []
        self.decoder_inputs = []
        self.target_weights = []
        for i in xrange(self.buckets[-1][0]):  # Last bucket is the biggest one.
            self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="encoder{0}".format(i)))
        for i in xrange(self.buckets[-1][1] + 1):
            self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="decoder{0}".format(i)))
            self.target_weights.append(tf.placeholder(tf.float32, shape=[None],
                                                name="weight{0}".format(i)))
        self.reward = [tf.placeholder(tf.float32, name="reward_%i" % i) for i in range(len(self.buckets))] # ADD
        
        
        # Our targets are decoder inputs shifted by one.
        targets = [self.decoder_inputs[i + 1] for i in xrange(len(self.decoder_inputs) - 1)]

        self.outputs, self.losses,self.encoder_state  = model_with_buckets(
                  self.encoder_inputs, self.decoder_inputs, targets,
                  self.target_weights, self.buckets, source_vocab_size, self.batch_size,
                  lambda x, y: seq2seq_f(x, y, tf.where(self.forward_only, True, False)),
                  output_projection=output_projection,
                  softmax_loss_function=softmax_loss_function)
        
        for b in xrange(len(self.buckets)):
            self.outputs[b] = [
                tf.cond(
                    self.forward_only,
                    lambda: tf.matmul(output, output_projection[0]) + output_projection[1],
                    lambda: output
                )
                for output in self.outputs[b]
            ]
        
        if not forward_only:
            with tf.name_scope("gradient_descent"):
                self.gradient_norms = []
                self.updates = []
                self.aj_losses = []
                self.gen_params = [p for p in tf.trainable_variables() if name_scope in p.name]
                opt = tf.train.AdamOptimizer()
                for b in xrange(len(self.buckets)):
                    R =  tf.subtract(self.reward[b], self.reward_bias)
                    adjusted_loss = tf.cond(self.up_reward,
                                              lambda:tf.multiply(self.losses[b], self.reward[b]),
                                              lambda: self.losses[b])

                    self.aj_losses.append(adjusted_loss)
                    gradients = tf.gradients(adjusted_loss, self.gen_params)
                    clipped_gradients, norm = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
                    self.gradient_norms.append(norm)
                    self.updates.append(opt.apply_gradients(
                        zip(clipped_gradients, self.gen_params), global_step=self.global_step))

        self.gen_variables = [k for k in tf.global_variables() if name_scope in k.name]
        self.saver = tf.train.Saver(tf.global_variables())

    def step(self, session, encoder_inputs, decoder_inputs, target_weights,
           bucket_id, forward_only=True, reward=1, mc_search=False, up_reward=False, debug=True):
        #print("model.step")
        # Check if the sizes match.
        encoder_size, decoder_size = self.buckets[bucket_id]
        if len(encoder_inputs) != encoder_size:
            raise ValueError("Encoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(encoder_inputs), encoder_size))
        if len(decoder_inputs) != decoder_size:
            raise ValueError("Decoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(decoder_inputs), decoder_size))
        if len(target_weights) != decoder_size:
            raise ValueError("Weights length must be equal to the one in bucket,"
                       " %d != %d." % (len(target_weights), decoder_size))
        #print("input feed")
        # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
        input_feed = {self.forward_only.name: forward_only,
                      self.up_reward.name:  up_reward,
                      self.mc_search.name: mc_search
                     }
        for l in xrange(len(self.buckets)):
            input_feed[self.reward[l].name] = reward
        for l in xrange(encoder_size):
            input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
        for l in xrange(decoder_size):
            input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
            input_feed[self.target_weights[l].name] = target_weights[l]
        #print("last_target")
        # Since our targets are decoder inputs shifted by one, we need one more.
        last_target = self.decoder_inputs[decoder_size].name
        input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)

        # Output feed: depends on whether we do a backward step or not.
        if not forward_only:
            #print("not forward_only")
            output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                     self.gradient_norms[bucket_id],  # Gradient norm.
                     self.losses[bucket_id]]  # Loss for this batch.
        else:
            #print("forward_only")
            output_feed = [self.losses[bucket_id]]  # Loss for this batch.
            for l in xrange(decoder_size):  # Output logits.
                output_feed.append(self.outputs[bucket_id][l])

        outputs = session.run(output_feed, input_feed)
        if not forward_only:
              return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
        else:
              return None, outputs[0], outputs[1:]  # No gradient norm, loss, outputs.

    def get_batch(self, data, bucket_id):

        encoder_size, decoder_size = self.buckets[bucket_id]
        encoder_inputs, decoder_inputs = [], []

        for _ in xrange(self.batch_size):
            encoder_input, decoder_input = random.choice(data[bucket_id])

            # Encoder inputs are padded and then reversed.
            encoder_pad = [PAD_ID] * (encoder_size - len(encoder_input))
            encoder_inputs.append(list(reversed(encoder_input + encoder_pad)))

            # Decoder inputs get an extra "GO" symbol, and are padded then.
            decoder_pad_size = decoder_size - len(decoder_input) - 1
            decoder_inputs.append([GO_ID] + decoder_input +
                            [PAD_ID] * decoder_pad_size)

        # Now we create batch-major vectors from the data selected above.
        batch_encoder_inputs, batch_decoder_inputs, batch_weights = [], [], []

        # Batch encoder inputs are just re-indexed encoder_inputs.
        for length_idx in xrange(encoder_size):
            batch_encoder_inputs.append(
                np.array([encoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

        # Batch decoder inputs are re-indexed decoder_inputs, we create weights.
        for length_idx in xrange(decoder_size):
            batch_decoder_inputs.append(
                np.array([decoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

            # Create target_weights to be 0 for targets that are padding.
            batch_weight = np.ones(self.batch_size, dtype=np.float32)
            for batch_idx in xrange(self.batch_size):
                # We set weight to 0 if the corresponding target is a PAD symbol.
                # The corresponding target is decoder_input shifted by 1 forward.
                if length_idx < decoder_size - 1:
                    target = decoder_inputs[batch_idx][length_idx + 1]
                if length_idx == decoder_size - 1 or target == PAD_ID:
                    batch_weight[batch_idx] = 0.0
            batch_weights.append(batch_weight)
        return batch_encoder_inputs, batch_decoder_inputs, batch_weights, encoder_inputs,decoder_inputs

In [14]:
def read_data(config):

    train_data_set = [[] for _ in config.buckets]
    dev_data_set = [[] for _ in config.buckets]

    s2s_data = pd.read_csv('_news_data.csv')
    train_data = s2s_data[0:900000]
    dev_data = s2s_data[900000:1000000]
    
    train_source = train_data['news_content']
    train_target = train_data['news_summary']
    
    dev_source = dev_data['news_content']
    dev_target = dev_data['news_summary']
    
    vocabulary = json.load(open('newdataset_vocabulary.json','r'))
    
    for m in range(0,len(train_source)):
        if m % 200000 == 0:
            print("  reading train data line %d" % m)
            sys.stdout.flush()
            
        source_sen = train_source[m]
        target_sen = train_target[m]
            
        source_word = []
        target_word = []
        
        try:
            s_w = source_sen.split(' ')
        except:
            s_w = ['float']
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            source_word.append(s_w[i])
            
        s_w = target_sen.split(' ')
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            target_word.append(s_w[i])            
            
        source_ids = []
        target_ids = []
            
        # convert to number
        
        for w in source_word:
            try:
                source_ids.append(vocabulary[w])
            except:
                source_ids.append(3)# 3 is UNK token
                
        # add start tag to target sentence
        #target_ids.append(1) # 1 is _GO: start token
  
        for w in target_word:
            try:
                target_ids.append(vocabulary[w])
            except:
                target_ids.append(3)# 3 is UNK token

        # add finish tag to target sentence. 
        target_ids.append(EOS_ID) # EOS_ID is end token
        
        
        # bucket data
        #config.buckets:  [bucket_id, (source_size, target_size)]
        for bucket_id, (source_size, target_size) in enumerate(config.buckets): #[bucket_id, (source_size, target_size)]
            if len(source_ids) < source_size and len(target_ids) < target_size:
                train_data_set[bucket_id].append([source_ids, target_ids])
                break
                 
    for m in range(900000,900000+len(dev_source)):
        if m % 200000 == 0:
            print("  reading dev data line %d" % m)
            sys.stdout.flush()
            
        source_sen = dev_source[m]
        target_sen = dev_target[m]
 
        source_word = []
        target_word = []
        
        try:
            s_w = source_sen.split(' ')
        except:
            s_w = ['float']
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            source_word.append(s_w[i])
            
        s_w = target_sen.split(' ')
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            target_word.append(s_w[i])    
            
        source_ids = []
        target_ids = []
            
        # convert to number
        for w in source_word:
            try:
                source_ids.append(vocabulary[w])
            except:
                source_ids.append(3)
                
        #target_ids.append(1) # 1 is _GO: start token
        for w in target_word:
            try:
                target_ids.append(vocabulary[w])
            except:
                target_ids.append(3)

        # add finish tag to target sentence. 
        target_ids.append(EOS_ID) # End token
                
        # bucket data
        #config.buckets:  [bucket_id, (source_size, target_size)]
        for bucket_id, (source_size, target_size) in enumerate(config.buckets): #[bucket_id, (source_size, target_size)]
            if len(source_ids) < source_size and len(target_ids) < target_size:
                dev_data_set[bucket_id].append([source_ids, target_ids])
                break
                                
    return vocabulary,train_data_set,dev_data_set
    #,train_query,dev_query



In [15]:
def prepare_data(gen_config):
    vocab, train_set,dev_set = read_data(gen_config)
    rev_vocab = {v: k for k, v in vocab.items()}
    return vocab, rev_vocab, dev_set, train_set
    #,train_query,dev_query

In [16]:
def create_model(session, gen_config, forward_only,name_scope, initializer=None):
    """Create translation model and initialize or load parameters in session."""
    with tf.variable_scope(name_or_scope=name_scope, initializer=initializer):
        model = Seq2SeqModel(gen_config, name_scope=name_scope, forward_only=forward_only)
        
        # checkpoint
        gen_ckpt_dir = os.path.abspath(os.path.join(gen_config.train_dir, "checkpoints"))
        # os.path.abspath: 返回path规范化的绝对路径。 
        ckpt = tf.train.get_checkpoint_state(gen_ckpt_dir)
        
        if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
            print("Reading Gen model parameters from %s" % ckpt.model_checkpoint_path)
            model.saver.restore(session, ckpt.model_checkpoint_path)
            #return model
        else:
            print("Created Gen model with fresh parameters.")
            gen_global_variables = [gv for gv in tf.global_variables() if name_scope in gv.name]
            session.run(tf.variables_initializer(gen_global_variables))
        return model

In [17]:
def train(gen_config):
    
    vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
    
    for b_set in train_set:
        print("bucket_set: ", len(b_set))

    with tf.Session() as sess:
    #with tf.device("/gpu:1"):
        # Create model.
        print("Creating %d layers of %d units." % (gen_config.num_layers, gen_config.emb_dim))
        model = create_model(sess, gen_config, forward_only=False,name_scope=gen_config.name_model)
        
        #size of each bucket in the train_dataset.
        train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
        # add them to get total train dataset size
        train_total_size = float(sum(train_bucket_sizes))
        # 0~1 --> id of bucket data.
        train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                               for i in xrange(len(train_bucket_sizes))]

        # This is the training loop.
        step_time, loss = 0.0, 0.0
        current_step = 0
        previous_losses = []
        gen_loss_summary = tf.Summary()
        gen_writer = tf.summary.FileWriter(gen_config.tensorboard_dir, sess.graph)
        print("training.......")
        while True:
            # Choose a bucket.
            # Choose a bucket according to data distribution. We pick a random number
            # in [0, 1] and use the corresponding interval in train_buckets_scale.
            random_number_01 = np.random.random_sample()
            bucket_id = min([i for i in xrange(len(train_buckets_scale)) if train_buckets_scale[i] > random_number_01])

            
            # Get a batch and make a step.
            start_time = time.time()
            encoder_inputs, decoder_inputs, target_weights,_,_ = model.get_batch(train_set, bucket_id)
            # model.get_batch 和 model.step 都是seq2seq里的函数。
            _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, forward_only=False)

            step_time += (time.time() - start_time) / gen_config.steps_per_checkpoint
            loss += step_loss / gen_config.steps_per_checkpoint
            current_step += 1

            
            
            if current_step % gen_config.steps_per_checkpoint == 0:
                bucket_value = gen_loss_summary.value.add()
                bucket_value.tag = gen_config.name_loss
                bucket_value.simple_value = float(loss)
                gen_writer.add_summary(gen_loss_summary, int(model.global_step.eval()))

                # Print statistics for the previous epoch.
                perplexity = math.exp(loss) if loss < 300 else float('inf')
                print ("global step %d learning rate %.4f step-time %.2f perplexity "
                       "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                                 step_time, perplexity))

                # Decrease learning rate if no improvement was seen over last 3 times.
                if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
                    sess.run(model.learning_rate_decay_op)
                previous_losses.append(loss)
                
                if current_step % (gen_config.steps_per_checkpoint * 3) == 0:
                    print("current_step: %d, save model" %(current_step))
                    gen_ckpt_dir = os.path.abspath(os.path.join(gen_config.train_dir, "checkpoints"))
                    if not os.path.exists(gen_ckpt_dir):
                        #os.makedirs() 方法用于递归创建目录
                        os.makedirs(gen_ckpt_dir)
                    checkpoint_path = os.path.join(gen_ckpt_dir, "chitchat.model")
                    model.saver.save(sess, checkpoint_path, global_step=model.global_step)

                step_time, loss = 0.0, 0.0
                sys.stdout.flush()



In [18]:
class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 64
    emb_dim = 256
    num_layers = 1
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_RNN1/"
    tensorboard_dir = "./1RNN_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(125,10),(125,12),(125,15),(125,30)]
    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]

In [19]:
#train(gen_config)

## decode (generate data for discriminator)

In [20]:
class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 1
    emb_dim = 256
    num_layers = 1
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_RNN1/"
    tensorboard_dir = "./1RNN_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(125,10),(125,12),(125,15),(125,30)]
    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]

In [21]:
def decode(decode_num_step):
    
    # Load vocabularies.
    vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
        
        
    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
    train_total_size = float(sum(train_bucket_sizes))
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]
    
    input_document = []
    target_summary = []
    generated_summary = []
    with tf.Session() as sess:
        # Create model and load parameters.
        model = create_model(sess, gen_config, forward_only=True, name_scope=gen_config.name_model)

        num_step = 0
        while num_step < decode_num_step:
            print("generating num_step: ", num_step)
            random_number_01 = np.random.random_sample()
            bucket_id = min([i for i in xrange(len(train_buckets_scale))
                             if train_buckets_scale[i] > random_number_01])
            # Get a 1-element batch to feed the sentence to the model.
            encoder_inputs, decoder_inputs, target_weights, _encoder_inputs, _decoder_inputs = model.get_batch(
                  train_set, bucket_id)#get_batch(train_set,bucket_id)
            # Get output logits for the sentence.
            _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)
            # This is a greedy decoder - outputs are just argmaxes of output_logits.
            outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
            #print('batch_encoder_input')
            #print(" ".join([str(rev_vocab[an]) for an in encoder_inputs]))
            # If there is an EOS symbol in outputs, cut them at that point.
            if EOS_ID in outputs:
                outputs = outputs[:outputs.index(EOS_ID)]
            # Print out French sentence corresponding to outputs.
            summary_g = " ".join([tf.compat.as_str(rev_vocab[output]) for output in outputs])
            print(summary_g)
            generated_summary.append(summary_g)
            
            for query, answer,outputs in zip(_encoder_inputs, _decoder_inputs,outputs):

                answer_str = " ".join([str(rev_vocab[an]) for an in answer])
                answer_str = answer_str.replace('_GO ','')
                answer_str = answer_str.replace(' _EOS','')
                answer_str = answer_str.replace(' _PAD','')
                print(answer_str)
                target_summary.append(answer_str)
                
                query_str = " ".join([str(rev_vocab[qu]) for qu in query])
                query_str = query_str.replace(' _PAD','')
                #print(query_str)
                document_str = query_str.split(' ')
                i = len(document_str)-1
                d_str = ''
                while i>0:
                    d_str = d_str + document_str[i]
                    d_str = d_str + ' '
                    i = i-1
                #print(d_str)
                input_document.append(d_str)
            num_step +=1
    return input_document, target_summary, generated_summary

In [22]:
def generated_data_for_D(Flag,decode_num_step):
    list1,list2,list3 = decode(decode_num_step)
    with open('generated_sample.txt','w')as f:
        for i in range(0,len(list3)):
            f.write(list3[i])
            f.write('\n')
    with open('real_sample.txt','w')as f:
        for i in range(0,len(list3)):
            f.write(list2[i])
            f.write('\n')
    with open('document_sample.txt','w')as f:
        for i in range(0,len(list1)):
            f.write(list1[i])
            f.write('\n')
    if Flag == True:
        with open('generated_data_for_D.csv', 'w') as datacsv:
            writer = csv.writer(datacsv,dialect=("excel"))
            writer.writerow(['generated_id','document_content','target_summary','generated_summary'])
            for i in range(0,len(list1)):
                writer.writerow([i,list1[i],list2[i],list3[i]])

In [23]:
#data = pd.read_csv('generated_data_for_D.csv')

In [24]:
#list1,list2,list3 = decode(20)

In [25]:
#data[0:10]

#### 写入real_sample.txt 和 generated_sample.txt 中

In [26]:
# experiment CNN 输入：
# - document + summary 判断
# - summary 判断。
# 看哪个结果好。
# 先试summary

In [27]:
#Flag = True # Flag = true: write a csv file Flag = False, don't write
#Flag = False
#decode_num_step = 5000
#generated_data_for_D(Flag,decode_num_step)

In [28]:
## Rouge score

In [29]:
#list1 = []
#with open("generated_sample.txt","r") as f:
#    for line in f:
#        list1.append(line)

In [30]:
#list1[0]

In [31]:
#list2 = []
#with open("real_sample.txt","r") as f:
#    for line in f:
#        list2.append(line)

In [32]:
#list2[0]

In [33]:
#from rouge import Rouge

In [34]:
'''rouge_1_f_sum = 0
rouge_1_p_sum = 0
rouge_1_r_sum = 0
rouge_2_f_sum = 0
rouge_2_p_sum = 0
rouge_2_r_sum = 0
rouge_l_f_sum = 0
rouge_l_p_sum = 0
rouge_l_r_sum = 0
num = 0
for i in range(0,len(list1)):
    hypothesis = list1[i]
    reference = list2[i]
    rouge = Rouge()
    scores = rouge.get_scores(reference, hypothesis)
    rouge_1_f = scores[0]['rouge-1']['f']
    rouge_1_p = scores[0]['rouge-1']['p']
    rouge_1_r = scores[0]['rouge-1']['r']
    rouge_2_f = scores[0]['rouge-2']['f']
    rouge_2_p = scores[0]['rouge-2']['p']
    rouge_2_r = scores[0]['rouge-2']['r']
    rouge_l_f = scores[0]['rouge-l']['f']
    rouge_l_p = scores[0]['rouge-l']['p']
    rouge_l_r = scores[0]['rouge-l']['r']
    
    rouge_1_f_sum = rouge_1_f_sum + rouge_1_f
    rouge_1_p_sum = rouge_1_p_sum + rouge_1_p
    rouge_1_r_sum = rouge_1_r_sum + rouge_1_r
    rouge_2_f_sum = rouge_2_f_sum + rouge_2_f
    rouge_2_p_sum = rouge_2_p_sum + rouge_2_p
    rouge_2_r_sum = rouge_2_r_sum + rouge_2_r
    rouge_l_f_sum = rouge_l_f_sum + rouge_l_f
    rouge_l_p_sum = rouge_l_p_sum + rouge_l_p
    rouge_l_r_sum = rouge_l_r_sum + rouge_l_r
    
    num = num + 1

rouge_1_F = rouge_1_f_sum/num
rouge_1_P = rouge_1_p_sum/num
rouge_1_R = rouge_1_r_sum/num
rouge_2_F = rouge_2_f_sum/num
rouge_2_P = rouge_2_p_sum/num
rouge_2_R = rouge_2_r_sum/num
rouge_l_F = rouge_l_f_sum/num
rouge_l_P = rouge_l_p_sum/num
rouge_l_R = rouge_l_r_sum/num'''

"rouge_1_f_sum = 0\nrouge_1_p_sum = 0\nrouge_1_r_sum = 0\nrouge_2_f_sum = 0\nrouge_2_p_sum = 0\nrouge_2_r_sum = 0\nrouge_l_f_sum = 0\nrouge_l_p_sum = 0\nrouge_l_r_sum = 0\nnum = 0\nfor i in range(0,len(list1)):\n    hypothesis = list1[i]\n    reference = list2[i]\n    rouge = Rouge()\n    scores = rouge.get_scores(reference, hypothesis)\n    rouge_1_f = scores[0]['rouge-1']['f']\n    rouge_1_p = scores[0]['rouge-1']['p']\n    rouge_1_r = scores[0]['rouge-1']['r']\n    rouge_2_f = scores[0]['rouge-2']['f']\n    rouge_2_p = scores[0]['rouge-2']['p']\n    rouge_2_r = scores[0]['rouge-2']['r']\n    rouge_l_f = scores[0]['rouge-l']['f']\n    rouge_l_p = scores[0]['rouge-l']['p']\n    rouge_l_r = scores[0]['rouge-l']['r']\n    \n    rouge_1_f_sum = rouge_1_f_sum + rouge_1_f\n    rouge_1_p_sum = rouge_1_p_sum + rouge_1_p\n    rouge_1_r_sum = rouge_1_r_sum + rouge_1_r\n    rouge_2_f_sum = rouge_2_f_sum + rouge_2_f\n    rouge_2_p_sum = rouge_2_p_sum + rouge_2_p\n    rouge_2_r_sum = rouge_2_r_su

In [35]:
'''print("rouge_1: f: ")
print(rouge_1_F)
print("rouge_1: p: ")
print(rouge_1_P)
print("rouge_1: r: ")
print(rouge_1_R)

#print("rouge_2: f: ")
#print(rouge_2_F)
#print("rouge_2: p: ")
#print(rouge_2_P)
#print("rouge_2: r: ")
#print(rouge_2_R)
      
print("rouge_L: f: ")
print(rouge_l_F)
print("rouge_L: p: ")
print(rouge_l_P)
print("rouge_L: r: ")
print(rouge_l_R)'''

'print("rouge_1: f: ")\nprint(rouge_1_F)\nprint("rouge_1: p: ")\nprint(rouge_1_P)\nprint("rouge_1: r: ")\nprint(rouge_1_R)\n\n#print("rouge_2: f: ")\n#print(rouge_2_F)\n#print("rouge_2: p: ")\n#print(rouge_2_P)\n#print("rouge_2: r: ")\n#print(rouge_2_R)\n      \nprint("rouge_L: f: ")\nprint(rouge_l_F)\nprint("rouge_L: p: ")\nprint(rouge_l_P)\nprint("rouge_L: r: ")\nprint(rouge_l_R)'

# pre-train discriminator

In [36]:
vocabulary = json.load(open('newdataset_vocabulary.json','r'))

In [37]:
class Dis_dataloader():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.sentences = np.array([])
        self.labels = np.array([])

    def load_train_data(self,positive_file, negative_file):
        # Load data
        positive_examples = []
        negative_examples = []
        
        positive_examples_word = []
        positive_examples = list(codecs.open(positive_file, "r", "utf-8").readlines())
        for s in positive_examples:
            #p_w = re.split(r'(;|,|\s)\s*', s)
            p_w = s.split()
            positive_examples_word.append(p_w)
                
        negative_examples_word = []
        negative_examples = list(codecs.open(negative_file, "r", "utf-8").readlines())
        for s in negative_examples:
            #n_w = re.split(r'(;|,|\s)\s*', s)
            #n_w = n_w.remove(' ')
            n_w = s.split()
            negative_examples_word.append(n_w)        
        
        
        # use part of data to test the model
        #positive_examples_word = positive_examples_word[:5000]
        #negative_examples_word = negative_examples_word[:5000]
        
        #Pads all sentences to the same length.#这里需要改为 全都变为一个长度 比如400之类的 然后把dis传入的参数
        #里的 senquence length改为一样的。
        padding_word="<PAD/>"

        positive_length = max(len(x) for x in positive_examples_word)
        negative_length = max(len(x) for x in negative_examples_word)
        if positive_length > negative_length:
            sequence_length = positive_length
        else:
            sequence_length = negative_length
            
        padded_positive_examples = []
        for i in range(len(positive_examples_word)):
            sentence = positive_examples_word[i]
            num_padding = sequence_length - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
            padded_positive_examples.append(new_sentence)
        
        padded_negative_examples = []
        for i in range(len(negative_examples_word)):
            sentence = negative_examples_word[i]
            num_padding = sequence_length - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
            padded_negative_examples.append(new_sentence)
        
        self.sentences = padded_positive_examples + padded_negative_examples
        
        # Generate labels
        positive_labels = [[0, 1] for _ in positive_examples_word]
        negative_labels = [[1, 0] for _ in negative_examples_word]
        #############################################################################
        self.labels = np.concatenate([positive_labels, negative_labels], 0)

        a = self.sentences
        b = self.labels
        
        a,b = shuffle(a,b)
        
        self.sentences = a
        self.lables = b
        
        #############################################################################
        sentences = self.sentences
        for sentence in sentences:
            for i in range(0,len(sentence)):
                sentence[i] = sentence[i].replace('.','')
                sentence[i] = sentence[i].replace(',','')
                sentence[i] = sentence[i].replace('?','')
                sentence[i] = sentence[i].replace('(','')
                sentence[i] = sentence[i].replace(')','')
                sentence[i] = sentence[i].replace('!','')
                sentence[i] = sentence[i].lower()
                
        for sentence in sentences:
            for i in range(0,len(sentence)):
                try: 
                    sentence[i] = vocabulary[sentence[i]]
                except: 
                    sentence[i] = 3 # len(vocabulary) + 1
        self.sentences = np.array(sentences)
        
        # Split batches
        self.num_batch = int(len(self.labels) / self.batch_size)
        self.sentences = self.sentences[:self.num_batch * self.batch_size]
        self.labels = self.labels[:self.num_batch * self.batch_size]
        self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
        self.labels_batches = np.split(self.labels, self.num_batch, 0)

        self.pointer = 0


    def next_batch(self):
        ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0


In [38]:
# An alternative to tf.nn.rnn_cell._linear function, which has been removed in Tensorfow 1.0.1
# The highway layer is borrowed from https://github.com/mkroutikov/tf-lstm-char-cnn
def dis_linear(input_, output_size, scope=None):
    '''
    Linear map: output[k] = sum_i(Matrix[k, i] * input_[i] ) + Bias[k]
    Args:
    input_: a tensor or a list of 2D, batch x n, Tensors.
    output_size: int, second dimension of W[i].
    scope: VariableScope for the created subgraph; defaults to "Linear".
  Returns:
    A 2D Tensor with shape [batch x output_size] equal to
    sum_i(input_[i] * W[i]), where W[i]s are newly created matrices.
  Raises:
    ValueError: if some of the arguments has unspecified or wrong shape.
  '''

    shape = input_.get_shape().as_list()
    if len(shape) != 2:
        raise ValueError("Linear is expecting 2D arguments: %s" % str(shape))
    if not shape[1]:
        raise ValueError("Linear expects shape[1] of arguments: %s" % str(shape))
    input_size = shape[1]

    # Now the computation.
    with tf.variable_scope(scope or "SimpleLinear"):
        matrix = tf.get_variable("Matrix", [output_size, input_size], dtype=input_.dtype)
        bias_term = tf.get_variable("Bias", [output_size], dtype=input_.dtype)

    return tf.matmul(input_, tf.transpose(matrix)) + bias_term

In [39]:
def highway(input_, size, num_layers=1, bias=-2.0, f=tf.nn.relu, scope='Highway'):
    """Highway Network (cf. http://arxiv.org/abs/1505.00387).
    t = sigmoid(Wy + b)
    z = t * g(Wy + b) + (1 - t) * y
    where g is nonlinearity, t is transform gate, and (1 - t) is carry gate.
    """

    with tf.variable_scope(scope):
        for idx in range(num_layers):
            g = f(dis_linear(input_, size, scope='highway_lin_%d' % idx))

            t = tf.sigmoid(dis_linear(input_, size, scope='highway_gate_%d' % idx) + bias)

            output = t * g + (1. - t) * input_
            input_ = output

    return output


In [40]:
class Discriminator(object):
    '''A CNN for classification: 
    Distinguish between the ground truth answer and the generated answer
    CNN: embedding layer --> convolutional, max pooling-->softmax '''
    
    # parameters
    def __init__(
                self, 
                sequence_length, 
                num_classes, 
                vocab_size,
                embedding_size, 
                filter_sizes, 
                num_filters, 
                l2_reg_lambda=0.0):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        with tf.variable_scope('discriminator'):
            # Embedding layer
            with tf.device('/cpu:0'), tf.name_scope("embedding"):
                self.W = tf.Variable(
                    tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                    name="W")
                self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
                self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)


            # Create a convolution + maxpool layer for each filter size
            pooled_outputs = []
            for filter_size, num_filter in zip(filter_sizes, num_filters):
                with tf.name_scope("conv-maxpool-%s" % filter_size):
                    # Convolution Layer
                    filter_shape = [filter_size, embedding_size, 1, num_filter]
                    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                    b = tf.Variable(tf.constant(0.1, shape=[num_filter]), name="b")
                    conv = tf.nn.conv2d(
                        self.embedded_chars_expanded,
                        W,
                        strides=[1, 1, 1, 1],
                        padding="VALID",
                        name="conv")
                    # Apply nonlinearity
                    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                    # Maxpooling over the outputs
                    pooled = tf.nn.max_pool(
                        h,
                        ksize=[1, sequence_length - filter_size + 1, 1, 1],
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name="pool")
                    pooled_outputs.append(pooled)
            
            # Combine all the pooled features
            num_filters_total = sum(num_filters)
            self.h_pool = tf.concat(pooled_outputs, 3)
            self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

            # Add highway
            with tf.name_scope("highway"):
                self.h_highway = highway(self.h_pool_flat, self.h_pool_flat.get_shape()[1], 1, 0)

            # Add dropout
            with tf.name_scope("dropout"):
                self.h_drop = tf.nn.dropout(self.h_highway, self.dropout_keep_prob)
            
            
            # Final (unnormalized) scores and predictions
            with tf.name_scope("output"):
                W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
                l2_loss += tf.nn.l2_loss(W)
                l2_loss += tf.nn.l2_loss(b)
                self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
                self.ypred_for_auc = tf.nn.softmax(self.scores)
                self.predictions = tf.argmax(self.scores, 1, name="predictions")
                
            # CalculateMean cross-entropy loss
            with tf.name_scope("loss"):
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
                self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss                
            print(self.loss)
            
        self.params = [param for param in tf.trainable_variables() if 'discriminator' in param.name]
        d_optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = d_optimizer.compute_gradients(self.loss, self.params, aggregation_method=2)
        self.train_op = d_optimizer.apply_gradients(grads_and_vars)       

#  Discriminator  Hyper-parameters
#########################################################################################
dis_embedding_dim = 64
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64

vocab_size = 436922
BATCH_SIZE = 64

In [41]:
## pre train discriminator 在all train 里包含了。 所以这部分不用跑。

In [42]:
def dis_train():
    dis_data_loader = Dis_dataloader(BATCH_SIZE)
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())

    negative_file = 'generated_sample.txt'
    positive_file = 'real_sample.txt'
    
    dis_data_loader.load_train_data(positive_file, negative_file)
    
    
    # Train 10 epoch on the generated data and do this for 50 times
    global_step = 0
    for current_step in range(3):
    
        print("global_step: ")
        print(global_step)
        
        for _ in range(3):
            print(' epoch: ')
            print(_)
            dis_data_loader.reset_pointer()
            for it in range(dis_data_loader.num_batch):
                x_batch, y_batch = dis_data_loader.next_batch()
                feed = {
                    discriminator.input_x: x_batch,
                    discriminator.input_y: y_batch,
                    discriminator.dropout_keep_prob: dis_dropout_keep_prob
                }
                _ = sess.run(discriminator.train_op, feed)
            global_step = global_step + 1
        #print("current_step: %d, save model" %(current_step))
        #checkpoint_path = os.path.join('./model/path')
        #discriminator.saver.save(sess, checkpoint_path)

In [43]:
## 尝试用disc model 得到reward

# GAN

In [44]:
# prepare disc_data for discriminator and generator
def disc_train_data(sess, gen_model, vocab, source_inputs, source_outputs,
                    encoder_inputs, decoder_inputs, target_weights, bucket_id, mc_search=False):
    print('disc_train_data method: ')
    train_query, train_answer_real = [], []
    query_len = gen_config.buckets[bucket_id][0]
    #answer_len = gen_config.buckets[bucket_id][1]
    answer_len = 30
    #print('try1')
    for query, answer in zip(source_inputs, source_outputs):
        query = query[:query_len] + [int(PAD_ID)] * (query_len - len(query) if query_len > len(query) else 0)
        train_query.append(query)
        answer = answer[:-1] # del tag EOS
        answer = answer[:answer_len] + [int(PAD_ID)] * (answer_len - len(answer) if answer_len > len(answer) else 0)
        train_answer_real.append(answer)
        train_labels = [1 for _ in source_inputs]
    #print(train_answer)
    
    train_answer = []
    def decoder(num_roll):
        output_list = []
        for _ in xrange(num_roll):
            print('roll_num')
            print(_)
            #print("try2")
            #_, _, output_logits = gen_model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id,
            #                                    forward_only=True, mc_search=mc_search)
            #print("try_f")
            _, _, output_logits = gen_model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id,True,mc_search = mc_search)
            
            seq_tokens = []
            resps = []
            for seq in output_logits:
                row_token = []
                for t in seq:
                    row_token.append(int(np.argmax(t, axis=0)))
                seq_tokens.append(row_token)

            seq_tokens_t = []
            for col in range(len(seq_tokens[0])):
                seq_tokens_t.append([seq_tokens[row][col] for row in range(len(seq_tokens))])

            for seq in seq_tokens_t:
                if EOS_ID in seq:
                    resps.append(seq[:seq.index(EOS_ID)][:gen_config.buckets[bucket_id][1]])
                else:
                    resps.append(seq[:gen_config.buckets[bucket_id][1]])
            #print(resps)
            
            for i, output in enumerate(resps):
                output = output[:answer_len] + [PAD_ID] * (answer_len - len(output) if answer_len > len(output) else 0)
                train_query.append(train_query[i])
                train_answer.append(output)
                train_labels.append(0)
                #print(train_answer)
        return train_query, train_answer, train_labels
    
    if mc_search:
        #print("try3")
        #train_query, train_answer, train_labels = decoder(gen_config.beam_size)
        #只decode一个结果，因为seq2seq模型decode出来结果一样。所以在reward函数里再做蒙特卡洛的生成text。
        train_query, train_answer, train_labels = decoder(1)
        #print("try4")
    else:
        train_query, train_answer, train_labels = decoder(1)
        #print("try5")

    return train_query, train_answer, train_labels,train_answer_real



In [45]:
'''# prepare disc_data for discriminator and generator
def disc_train_data(sess, gen_model, vocab,rev_vocab, source_inputs, source_outputs,
                    encoder_inputs, decoder_inputs, target_weights, bucket_id, mc_search=False):
    print('disc_train_data method: ')
    #train_query, train_answer = [], []
    query_len = gen_config.buckets[bucket_id][0]
    answer_len = gen_config.buckets[bucket_id][1]

    train_answer = []
    train_query = []
    train_labels = []
    def decoder(num_roll):
        output_list = []
        for _ in xrange(num_roll):
            print('roll_num')
            print(_)
            #print("try2")
            _, _, output_logits = gen_model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)

            outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
            if EOS_ID in outputs:
                outputs = outputs[:outputs.index(EOS_ID)]
            
            # outputs
            summary_g = " ".join([tf.compat.as_str(rev_vocab[output]) for output in outputs])
            #generated_summary.append(summary_g)
            
            for query, answer,outputs in zip(source_inputs, source_outputs,outputs):

                answer_str = " ".join([str(rev_vocab[an]) for an in answer])
                answer_str = answer_str.replace('_GO ','')
                answer_str = answer_str.replace(' _EOS','')
                answer_str = answer_str.replace(' _PAD','')
                #print(answer_str)
                #target_summary.append(answer_str)
                
                query_str = " ".join([str(rev_vocab[qu]) for qu in query])
                query_str = query_str.replace(' _PAD','')
                #print(query_str)
                document_str = query_str.split(' ')
                i = len(document_str)-1
                d_str = ''
                while i>0:
                    d_str = d_str + document_str[i]
                    d_str = d_str + ' '
                    i = i-1
                #print(d_str)
                #input_document.append(d_str)
            
            print("document")
            print(d_str)
            print("target summary")
            print(answer_str)
            print("generated summary")
            print(summary_g)
            
            train_query.append(query)
            train_answer.append(outputs)
            train_labels.append(0)
            #train_query.append(query)
            #train_answer.append(answer)
            #train_labels.append(1)

        return train_query, train_answer, train_labels
    
    if mc_search:
        #print("try3")
        #train_query, train_answer, train_labels = decoder(gen_config.beam_size)
        train_query, train_answer, train_labels = decoder(1)
        #print("try4")
    else:
        train_query, train_answer, train_labels = decoder(1)
        #print("try5")

    return train_query, train_answer, train_labels

'''

'# prepare disc_data for discriminator and generator\ndef disc_train_data(sess, gen_model, vocab,rev_vocab, source_inputs, source_outputs,\n                    encoder_inputs, decoder_inputs, target_weights, bucket_id, mc_search=False):\n    print(\'disc_train_data method: \')\n    #train_query, train_answer = [], []\n    query_len = gen_config.buckets[bucket_id][0]\n    answer_len = gen_config.buckets[bucket_id][1]\n\n    train_answer = []\n    train_query = []\n    train_labels = []\n    def decoder(num_roll):\n        output_list = []\n        for _ in xrange(num_roll):\n            print(\'roll_num\')\n            print(_)\n            #print("try2")\n            _, _, output_logits = gen_model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)\n\n            outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]\n            if EOS_ID in outputs:\n                outputs = outputs[:outputs.index(EOS_ID)]\n            \n            # outputs\n 

In [46]:
def get_reward(sess, input_x, rollout_num, bucket_id, discriminator,source_inputs,source_outputs):
    rewards = []
    
    self_batch_size = len(input_x)
    self_sequence_length = gen_config.buckets[bucket_id][1]
    
    self_x = tf.placeholder(tf.int32, shape=[self_batch_size, self_sequence_length])
    self_x = input_x

    feed = {discriminator.input_x: self_x, discriminator.dropout_keep_prob: 1.0}
    ypred_for_auc = sess.run(discriminator.ypred_for_auc, feed)
    print(ypred_for_auc)
    ypred = np.array([item[1] for item in ypred_for_auc])
    print(ypred)
    reward = ypred[0]
    rewards = 1-reward
    print(rewards)
    return rewards

In [47]:
#  Discriminator  Hyper-parameters
#########################################################################################
dis_embedding_dim = 64
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64

vocab_size = 436922
BATCH_SIZE = 64
vocab_size = 250000

In [61]:
def all_train():
    dis_data_loader = Dis_dataloader(BATCH_SIZE)

    sess = tf.Session()
    
    gen_model = create_model(sess, gen_config, forward_only=False, name_scope=gen_config.name_model)
    discriminator = Discriminator(sequence_length=30, num_classes=2, vocab_size=vocab_size, embedding_size=dis_embedding_dim, 
                                filter_sizes=dis_filter_sizes, num_filters=dis_num_filters, l2_reg_lambda=dis_l2_reg_lambda)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess.run(tf.global_variables_initializer())
    
    # Load vocabularies.
    vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
    
    #for set in train_set:
    #   print("all train len: ", len(set))

    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
    train_total_size = float(sum(train_bucket_sizes))
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                               for i in xrange(len(train_bucket_sizes))]
    ######################################################################################################
    #generate samples for discriminator
    # generated_sample.txt and real_sample.txt
    print("generate samples for training discriminator")
    input_document = []
    target_summary = []
    generated_summary = []
    
    _decode_num_step = 1000
    num_step = 0
    while num_step < _decode_num_step:
        random_number_01 = np.random.random_sample()
        bucket_id = min([i for i in xrange(len(train_buckets_scale))
                             if train_buckets_scale[i] > random_number_01])
        # Get a 1-element batch to feed the sentence to the model.
        encoder_inputs, decoder_inputs, target_weights, _encoder_inputs, _decoder_inputs = gen_model.get_batch(
                  train_set, bucket_id)#get_batch(train_set,bucket_id)
        # Get output logits for the sentence.
        _, _, output_logits = gen_model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)
        # This is a greedy decoder - outputs are just argmaxes of output_logits.
        outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
        #print('batch_encoder_input')
        #print(" ".join([str(rev_vocab[an]) for an in encoder_inputs]))
        # If there is an EOS symbol in outputs, cut them at that point.
        if EOS_ID in outputs:
            outputs = outputs[:outputs.index(EOS_ID)]
        # Print out French sentence corresponding to outputs.
        summary_g = " ".join([tf.compat.as_str(rev_vocab[output]) for output in outputs])
        #print(summary_g)
        generated_summary.append(summary_g)
            
        for query, answer,outputs in zip(_encoder_inputs, _decoder_inputs,outputs):

            answer_str = " ".join([str(rev_vocab[an]) for an in answer])
            answer_str = answer_str.replace('_GO ','')
            answer_str = answer_str.replace(' _EOS','')
            answer_str = answer_str.replace(' _PAD','')
            #print(answer_str)
            target_summary.append(answer_str)
                
            query_str = " ".join([str(rev_vocab[qu]) for qu in query])
            query_str = query_str.replace(' _PAD','')
            #print(query_str)
            document_str = query_str.split(' ')
            i = len(document_str)-1
            d_str = ''
            while i>0:
                d_str = d_str + document_str[i]
                d_str = d_str + ' '
                i = i-1
            #print(d_str)
            input_document.append(d_str)
        num_step +=1

    with open('generated_sample.txt','w')as f:
        for i in range(0,len(generated_summary)):
            f.write(generated_summary[i])
            f.write('\n')
    f.close()
    
    with open('real_sample.txt','w')as f:
        for i in range(0,len(target_summary)):
            f.write(target_summary[i])
            f.write('\n')
    f.close()
    
    ######################################################################################################
    print('Start pre-training...')
    print('pre-training Discriminator')
    
    negative_file = 'generated_sample.txt'
    positive_file = 'real_sample.txt'
    dis_data_loader.load_train_data(positive_file, negative_file)
    
    
    # Train 5 epoch on the generated data and do this for 50 times
    global_step = 0
    for current_step in range(10):
    
        print("global_step: ")
        print(global_step)
        
        for _ in range(5):
            #print(' epoch: ')
            #print(_)
            dis_data_loader.reset_pointer()
            for it in range(dis_data_loader.num_batch):
                x_batch, y_batch = dis_data_loader.next_batch()
                feed = {
                    discriminator.input_x: x_batch,
                    discriminator.input_y: y_batch,
                    discriminator.dropout_keep_prob: dis_dropout_keep_prob
                }
                _ = sess.run(discriminator.train_op, feed)
            global_step = global_step + 1
    ######################################################################################
    
    #vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
    
    #for set in train_set:
    #   print("all train len: ", len(set))

    #train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
    #train_total_size = float(sum(train_bucket_sizes))
    #train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
    #                           for i in xrange(len(train_bucket_sizes))]
        
    ######################################################################################
    current_step = 0
    step_time, disc_loss, gen_loss, t_loss, batch_reward = 0.0, 0.0, 0.0, 0.0, 0.0
    gen_loss_summary = tf.Summary()
    gen_writer = tf.summary.FileWriter(gen_config.tensorboard_dir, sess.graph)

    while True:
        current_step += 1
        start_time = time.time()
        random_number_01 = np.random.random_sample()
        bucket_id = min([i for i in xrange(len(train_buckets_scale))
                        if train_buckets_scale[i] > random_number_01])
        
        print("==================Update Generator: %d=========================" % current_step)
        # 1.Sample (X,Y) from real disc_data
        update_gen_data = gen_model.get_batch(train_set, bucket_id)
        encoder, decoder, weights, source_inputs, source_outputs = update_gen_data 
        # 2.Sample (X,Y) and (X, ^Y) through ^Y ~ G(*|X) with Monte Carlo search
        train_query, train_answer, train_labels,answer_real = disc_train_data(sess, gen_model, vocab, source_inputs, source_outputs,
                                                                encoder, decoder, weights, bucket_id, mc_search=True)
        print("=============================mc_search: True====================================")   
        # 3.Compute Reward r for (X, ^Y ) using D.---based on Monte Carlo search
        #***********************************************************************#
        rollout_num = gen_config.beam_size
        print("train_answer")
        #train_answer = train_answer[1]
        print(train_answer)
        print('target answer')
        print(answer_real)
        #print(len(train_answer))
        reward = get_reward(sess, train_answer, rollout_num, bucket_id, discriminator,source_inputs,source_outputs)
        print(reward)
        
        print('=========================tuning reward==============================')
        batch_reward += reward / gen_config.steps_per_checkpoint
        print(batch_reward)
        #reward = 1
        #print(reward)
        #batch_reward += reward / gen_config.steps_per_checkpoint
        #print(batch_reward)

        print("step_reward: ", reward)
            
        #*************************************************************************#
        # 4.Update G on (X, ^Y ) using reward r
        gan_adjusted_loss, gen_step_loss, _ =gen_model.step(sess, encoder, decoder, weights, bucket_id, forward_only=True,
                                           reward=reward, up_reward=True, debug=True)
        #gan_adjusted_loss, gen_step_loss, _ =gen_model.step(sess, encoder, decoder, weights, bucket_id, forward_only=True)
        gen_loss += gen_step_loss / gen_config.steps_per_checkpoint

        print("gen_step_loss: ", gen_step_loss)
        print("gen_step_adjusted_loss: ", gan_adjusted_loss)

        # 5.Teacher-Forcing: Update G on (X, Y )
        t_adjusted_loss, t_step_loss, a = gen_model.step(sess, encoder, decoder, weights, bucket_id, forward_only=False)
        t_loss += t_step_loss / gen_config.steps_per_checkpoint
           
        print("t_step_loss: ", t_step_loss)
        print("t_adjusted_loss", t_adjusted_loss)           

        if current_step % gen_config.steps_per_checkpoint == 0:
            print('######################################################################')
            print('######################################################################')
            print('######################################################################')
            
            step_time += (time.time() - start_time) / gen_config.steps_per_checkpoint

            print("current_steps: %d, step time: %.4f, disc_loss: %.3f, gen_loss: %.3f, t_loss: %.3f, reward: %.3f"
                      %(current_step, step_time, disc_loss, gen_loss, t_loss, batch_reward))

            gen_global_steps = sess.run(gen_model.global_step)
            gen_loss_value = gen_loss_summary.value.add()
            gen_loss_value.tag = gen_config.name_loss
            gen_loss_value.simple_value = float(gen_loss)
            t_loss_value = gen_loss_summary.value.add()
            t_loss_value.tag = gen_config.teacher_loss
            t_loss_value.simple_value = float(t_loss)
            batch_reward_value = gen_loss_summary.value.add()
            batch_reward_value.tag = gen_config.reward_name
            batch_reward_value.simple_value = float(batch_reward)
            gen_writer.add_summary(gen_loss_summary, int(gen_global_steps))

            if current_step % (gen_config.steps_per_checkpoint * 2) == 0:
                print("current_steps: %d, save gen model" % current_step)
                gen_ckpt_dir = os.path.abspath(os.path.join(gen_config.train_dir, "checkpoints"))
                if not os.path.exists(gen_ckpt_dir):
                        os.makedirs(gen_ckpt_dir)
                gen_model_path = os.path.join(gen_ckpt_dir, "gen.model")
                gen_model.saver.save(sess, gen_model_path, global_step=gen_model.global_step)

            step_time, disc_loss, gen_loss, t_loss, batch_reward = 0.0, 0.0, 0.0, 0.0, 0.0
            sys.stdout.flush()
        if current_step % (3*gen_config.steps_per_checkpoint) == 0:
            print("======================update discriminator=====================")
                #generate samples for discriminator
                # generated_sample.txt and real_sample.txt
                print("generate samples for training discriminator")
                input_document = []
                target_summary = []
                generated_summary = []
    
                _decode_num_step = 1000
                num_step = 0
                while num_step < _decode_num_step:
                random_number_01 = np.random.random_sample()
                bucket_id = min([i for i in xrange(len(train_buckets_scale))
                             if train_buckets_scale[i] > random_number_01])
                # Get a 1-element batch to feed the sentence to the model.
                encoder_inputs, decoder_inputs, target_weights, _encoder_inputs, _decoder_inputs = gen_model.get_batch(
                          train_set, bucket_id)#get_batch(train_set,bucket_id)
                # Get output logits for the sentence.
                _, _, output_logits = gen_model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)
                # This is a greedy decoder - outputs are just argmaxes of output_logits.
                outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
                #print('batch_encoder_input')
                #print(" ".join([str(rev_vocab[an]) for an in encoder_inputs]))
                # If there is an EOS symbol in outputs, cut them at that point.
                if EOS_ID in outputs:
                    outputs = outputs[:outputs.index(EOS_ID)]
                # Print out French sentence corresponding to outputs.
                summary_g = " ".join([tf.compat.as_str(rev_vocab[output]) for output in outputs])
                #print(summary_g)
                generated_summary.append(summary_g)
            
                for query, answer,outputs in zip(_encoder_inputs, _decoder_inputs,outputs):

                    answer_str = " ".join([str(rev_vocab[an]) for an in answer])
                    answer_str = answer_str.replace('_GO ','')
                    answer_str = answer_str.replace(' _EOS','')
                    answer_str = answer_str.replace(' _PAD','')
                    #print(answer_str)
                    target_summary.append(answer_str)
                
                    query_str = " ".join([str(rev_vocab[qu]) for qu in query])
                    query_str = query_str.replace(' _PAD','')
                    #print(query_str)
                    document_str = query_str.split(' ')
                    i = len(document_str)-1
                    d_str = ''
                    while i>0:
                        d_str = d_str + document_str[i]
                        d_str = d_str + ' '
                        i = i-1
                    #print(d_str)
                    input_document.append(d_str)
                num_step +=1

            with open('generated_sample.txt','w')as f:
                for i in range(0,len(generated_summary)):
                    f.write(generated_summary[i])
                    f.write('\n')
            f.close()

            with open('real_sample.txt','w')as f:
                for i in range(0,len(target_summary)):
                    f.write(target_summary[i])
                    f.write('\n')
            f.close()

            ######################################################################################################
            print('Start pre-training...')
            print('pre-training Discriminator')

            negative_file = 'generated_sample.txt'
            positive_file = 'real_sample.txt'
            dis_data_loader.load_train_data(positive_file, negative_file)


            # Train 5 epoch on the generated data and do this for 50 times
            global_step = 0
            for current_step in range(10):

                print("global_step: ")
                print(global_step)

                for _ in range(5):
                    #print(' epoch: ')
                    #print(_)
                    dis_data_loader.reset_pointer()
                    for it in range(dis_data_loader.num_batch):
                        x_batch, y_batch = dis_data_loader.next_batch()
                        feed = {
                            discriminator.input_x: x_batch,
                            discriminator.input_y: y_batch,
                            discriminator.dropout_keep_prob: dis_dropout_keep_prob
                        }
                        _ = sess.run(discriminator.train_op, feed)
                    global_step = global_step + 1

IndentationError: unexpected indent (<ipython-input-61-0e1cbd2de4e6>, line 227)

In [49]:
#all_train()

## step 1: pre-train the generator

In [50]:
'''class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 64
    emb_dim = 256
    num_layers = 1
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_RNN1/"
    tensorboard_dir = "./1RNN_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(125,10),(125,12),(125,15),(125,30)]
    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]'''

'class gen_config(object):\n    beam_size = 10\n    learning_rate = 0.5\n    learning_rate_decay_factor = 0.99\n    max_gradient_norm = 2.0\n    batch_size = 64\n    emb_dim = 256\n    num_layers = 1\n    vocab_size = 250000\n    name_model = "st_model"\n    train_dir = "./gen_data_RNN1/"\n    tensorboard_dir = "./1RNN_tensorboard/gen_log/"\n    name_loss = "gen_loss"\n    teacher_loss = "teacher_loss"\n    reward_name = "reward"\n    max_train_data_size = 0\n    steps_per_checkpoint = 100\n    buckets = [(125,10),(125,12),(125,15),(125,30)]\n    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]'

In [51]:
# train(gen_config)

## step 2: generate samples using sequence to sequence model

In [52]:
'''class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 1
    emb_dim = 256
    num_layers = 1
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_RNN1/"
    tensorboard_dir = "./1RNN_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(125,10),(125,12),(125,15),(125,30)]
    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]'''

'class gen_config(object):\n    beam_size = 10\n    learning_rate = 0.5\n    learning_rate_decay_factor = 0.99\n    max_gradient_norm = 2.0\n    batch_size = 1\n    emb_dim = 256\n    num_layers = 1\n    vocab_size = 250000\n    name_model = "st_model"\n    train_dir = "./gen_data_RNN1/"\n    tensorboard_dir = "./1RNN_tensorboard/gen_log/"\n    name_loss = "gen_loss"\n    teacher_loss = "teacher_loss"\n    reward_name = "reward"\n    max_train_data_size = 0\n    steps_per_checkpoint = 100\n    buckets = [(125,10),(125,12),(125,15),(125,30)]\n    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]'

In [53]:
'''#Flag = True # Flag = true: write a csv file Flag = False, don't write
#Flag = False
#decode_num_step = 5000
#generated_data_for_D(Flag,decode_num_step)'''

"#Flag = True # Flag = true: write a csv file Flag = False, don't write\n#Flag = False\n#decode_num_step = 5000\n#generated_data_for_D(Flag,decode_num_step)"

## step 3: update discriminator: in all train part

## step 4: train GAN

In [54]:
#all_train()

## After training the GAN 

## test generator again

In [55]:
class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 1
    emb_dim = 256
    num_layers = 1
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_RNN1/"
    tensorboard_dir = "./1RNN_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(125,10),(125,12),(125,15),(125,30)]
    buckets_concat = [(125,10),(125,12),(125,15),(125,30)]

In [56]:
def decode(decode_num_step):
    
    # Load vocabularies.
    vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
        
        
    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
    train_total_size = float(sum(train_bucket_sizes))
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]
    
    input_document = []
    target_summary = []
    generated_summary = []
    with tf.Session() as sess:
        # Create model and load parameters.
        model = create_model(sess, gen_config, forward_only=True, name_scope=gen_config.name_model)

        num_step = 0
        while num_step < decode_num_step:
            print("generating num_step: ", num_step)
            random_number_01 = np.random.random_sample()
            bucket_id = min([i for i in xrange(len(train_buckets_scale))
                             if train_buckets_scale[i] > random_number_01])
            # Get a 1-element batch to feed the sentence to the model.
            encoder_inputs, decoder_inputs, target_weights, _encoder_inputs, _decoder_inputs = model.get_batch(
                  train_set, bucket_id)#get_batch(train_set,bucket_id)
            # Get output logits for the sentence.
            _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)
            # This is a greedy decoder - outputs are just argmaxes of output_logits.
            outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
            #print('batch_encoder_input')
            #print(" ".join([str(rev_vocab[an]) for an in encoder_inputs]))
            # If there is an EOS symbol in outputs, cut them at that point.
            if EOS_ID in outputs:
                outputs = outputs[:outputs.index(EOS_ID)]
            # Print out French sentence corresponding to outputs.
            summary_g = " ".join([tf.compat.as_str(rev_vocab[output]) for output in outputs])
            print(summary_g)
            generated_summary.append(summary_g)
            
            for query, answer,outputs in zip(_encoder_inputs, _decoder_inputs,outputs):

                answer_str = " ".join([str(rev_vocab[an]) for an in answer])
                answer_str = answer_str.replace('_GO ','')
                answer_str = answer_str.replace(' _EOS','')
                answer_str = answer_str.replace(' _PAD','')
                print(answer_str)
                target_summary.append(answer_str)
                
                query_str = " ".join([str(rev_vocab[qu]) for qu in query])
                query_str = query_str.replace(' _PAD','')
                #print(query_str)
                document_str = query_str.split(' ')
                i = len(document_str)-1
                d_str = ''
                while i>0:
                    d_str = d_str + document_str[i]
                    d_str = d_str + ' '
                    i = i-1
                print('document')
                print(d_str)
                print('################################################################')
                input_document.append(d_str)
            num_step +=1
    return input_document, target_summary, generated_summary

In [57]:
list1,list2,list3 = decode(2000)

  reading train data line 0
  reading train data line 200000
  reading train data line 400000
  reading train data line 600000
  reading train data line 800000
single_cell
Reading Gen model parameters from /Users/zhangyiman/final project/ Seq2Seq_GAN_Try_Again/gen_data_RNN1/checkpoints/chitchat.model-21300
generating num_step:  0
_UNK _UNK _UNK _UNK _UNK _UNK _UNK _UNK of the beast 
tatsumi _UNK – shoujo _UNK _UNK aka path of the beast 1980 
document
the post tatsumi _UNK – shoujo _UNK _UNK aka path of the beast 1980 appear first on _UNK com  the sizzl japanes tatsumi _UNK erot drama shoujo _UNK _UNK aka whore girl – the anim trail distribut intern as path of the _UNK offer a venu for sex kitten _UNK yoshimura to reveal choic area of her _UNK frame saki has drop out of school and when she not help her mother sell noodl in the street she play flesh game with boyfriend _UNK in her spare time the girl also entertain a truck driver name _UNK determin to improv her lot in life she doesn't w

In [58]:
from rouge import Rouge

In [59]:
rouge_1_f_sum = 0
rouge_1_p_sum = 0
rouge_1_r_sum = 0
rouge_2_f_sum = 0
rouge_2_p_sum = 0
rouge_2_r_sum = 0
rouge_l_f_sum = 0
rouge_l_p_sum = 0
rouge_l_r_sum = 0
num = 0
for i in range(0,len(list1)):
    hypothesis = list1[i]
    reference = list2[i]
    rouge = Rouge()
    scores = rouge.get_scores(reference, hypothesis)
    rouge_1_f = scores[0]['rouge-1']['f']
    rouge_1_p = scores[0]['rouge-1']['p']
    rouge_1_r = scores[0]['rouge-1']['r']
    rouge_2_f = scores[0]['rouge-2']['f']
    rouge_2_p = scores[0]['rouge-2']['p']
    rouge_2_r = scores[0]['rouge-2']['r']
    rouge_l_f = scores[0]['rouge-l']['f']
    rouge_l_p = scores[0]['rouge-l']['p']
    rouge_l_r = scores[0]['rouge-l']['r']
    
    rouge_1_f_sum = rouge_1_f_sum + rouge_1_f
    rouge_1_p_sum = rouge_1_p_sum + rouge_1_p
    rouge_1_r_sum = rouge_1_r_sum + rouge_1_r
    rouge_2_f_sum = rouge_2_f_sum + rouge_2_f
    rouge_2_p_sum = rouge_2_p_sum + rouge_2_p
    rouge_2_r_sum = rouge_2_r_sum + rouge_2_r
    rouge_l_f_sum = rouge_l_f_sum + rouge_l_f
    rouge_l_p_sum = rouge_l_p_sum + rouge_l_p
    rouge_l_r_sum = rouge_l_r_sum + rouge_l_r
    
    num = num + 1

rouge_1_F = rouge_1_f_sum/num
rouge_1_P = rouge_1_p_sum/num
rouge_1_R = rouge_1_r_sum/num
rouge_2_F = rouge_2_f_sum/num
rouge_2_P = rouge_2_p_sum/num
rouge_2_R = rouge_2_r_sum/num
rouge_l_F = rouge_l_f_sum/num
rouge_l_P = rouge_l_p_sum/num
rouge_l_R = rouge_l_r_sum/num

In [60]:
## print("rouge_1: f: ")
print(rouge_1_F)
print("rouge_1: p: ")
print(rouge_1_P)
print("rouge_1: r: ")
print(rouge_1_R)
print('cccc')
print(rouge_2_F)
print("rouge_2: p: ")
print(rouge_2_P)
print("rouge_2: r: ")
print(rouge_2_R)
      
    
print("rouge_L: f: ")
print(rouge_l_F)
print("rouge_L: p: ")
print(rouge_l_P)
print("rouge_L: r: ")
print(rouge_l_R)

0.21394564640189517
rouge_1: p: 
0.7216682596748359
rouge_1: r: 
0.13949425376076893
cccc
0.06781538442618044
rouge_2: p: 
0.29773399595697997
rouge_2: r: 
0.04071403200354495
rouge_L: f: 
0.07238321887765127
rouge_L: p: 
0.6406444225797593
rouge_L: r: 
0.07603910473748646
